In [2]:
%reload_ext autoreload
%autoreload 1
%aimport helper

In [2]:
import pandas as pd

In [3]:
data_crimes_train = pd.read_csv('train.csv')

In [3]:
data_crimes_train.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [16]:
data_crimes_train.describe()

,X,Y
count,878049.000000,878049.000000
mean,-122.422616,37.771020
std,0.030354,0.456893
min,-122.513642,37.707879
25%,-122.432952,37.752427
50%,-122.416420,37.775421
75%,-122.406959,37.784369
max,-120.500000,90.000000


In [20]:
data_crimes_train.Category.unique()

array(['WARRANTS', 'OTHER OFFENSES', 'LARCENY/THEFT', 'VEHICLE THEFT',
       'VANDALISM', 'NON-CRIMINAL', 'ROBBERY', 'ASSAULT', 'WEAPON LAWS',
       'BURGLARY', 'SUSPICIOUS OCC', 'DRUNKENNESS',
       'FORGERY/COUNTERFEITING', 'DRUG/NARCOTIC', 'STOLEN PROPERTY',
       'SECONDARY CODES', 'TRESPASS', 'MISSING PERSON', 'FRAUD',
       'KIDNAPPING', 'RUNAWAY', 'DRIVING UNDER THE INFLUENCE',
       'SEX OFFENSES FORCIBLE', 'PROSTITUTION', 'DISORDERLY CONDUCT',
       'ARSON', 'FAMILY OFFENSES', 'LIQUOR LAWS', 'BRIBERY',
       'EMBEZZLEMENT', 'SUICIDE', 'LOITERING', 'SEX OFFENSES NON FORCIBLE',
       'EXTORTION', 'GAMBLING', 'BAD CHECKS', 'TREA', 'RECOVERED VEHICLE',
       'PORNOGRAPHY/OBSCENE MAT'], dtype=object)

In [48]:
len(data_crimes_train.Address.unique())

23228

In [10]:
values = set(data_crimes_train.PdDistrict.values)
print(values)

{'INGLESIDE', 'CENTRAL', 'TARAVAL', 'BAYVIEW', 'SOUTHERN', 'PARK', 'MISSION', 'NORTHERN', 'TENDERLOIN', 'RICHMOND'}


In [4]:
def transformColumnToNumber(df,columnName):
    values = list(set(df[columnName].values))
    dictValues = {}
    for i in range(len(values)):
        dictValues[values[i]] = i
    
    df[columnName] = df[columnName].apply(lambda value: dictValues[value])
    return df

In [13]:
data_crimes_train_later = transformColumnToNumber(data_crimes_train,'DayOfWeek')
data_crimes_train_later = transformColumnToNumber(data_crimes_train_later,'PdDistrict')

In [14]:
data_crimes_train_later.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,0,4,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,0,4,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,0,4,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,0,4,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,0,2,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [15]:
feature_cols = ['DayOfWeek', 'PdDistrict'] #,'Address','X','Y']
# you want all rows, and the feature_cols' columns
X = data_crimes_train_later.loc[:, feature_cols]

In [16]:
X.shape

(878049, 2)

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X)
X_train_counts.shape

(6, 4)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
X_TFIDF = TfidfVectorizer().fit_transform(X)
X_TFIDF.shape

KeyboardInterrupt: 

In [17]:
# now we want to create our response vector
y = data_crimes_train.Category

In [8]:
y.shape

(878049,)

In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier

In [20]:
def run_model(clf, X, y):
    scores = cross_val_score(clf, X, y, cv=5)
    print("%s accuracy: %0.2f (+/- %0.2f)" % \
          (str(clf.__class__).split('.')[-1].replace('>','').replace("'",''), 
          scores.mean(), scores.std() * 2))
def run_models(X, y):
    run_model(LinearSVC(), X, y)
    run_model(SGDClassifier(), X, y)
    run_model(Perceptron(), X, y)
    run_model(PassiveAggressiveClassifier(), X, y)
    run_model(BernoulliNB(), X, y)
#     run_model(MultinomialNB(), X, y)
#     run_model(KNeighborsClassifier(), X, y)
    run_model(NearestCentroid(), X, y)
#     run_model(RandomForestClassifier(n_estimators=50, max_depth=5), X, y)

In [37]:
logreg = LinearSVC()
logreg.fit(X_TFIDF,y)

ValueError: Found input variables with inconsistent numbers of samples: [6, 878049]

In [18]:
run_model(LogisticRegression(), X, y)

LogisticRegression accuracy: 0.20 (+/- 0.00)


In [ ]:
run_models(X, y)

In [4]:
data_crimes_test = pd.read_csv('test.csv')
data_crimes_test.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
